# Repeating earlier kind of work flow with new dataset, taken from OpenNeuro

Contains T1w scans only of children, adolescents and adults. 


In [1]:
import os 
import glob
import os.path as op
from os.path import abspath

from nipype import Workflow, Node, MapNode, Function, IdentityInterface, DataGrabber
from nipype.interfaces.fsl import BET, ApplyMask, FAST, FLIRT, FNIRT, InvWarp, ApplyWarp

from nilearn.plotting import plot_anat
%matplotlib inline
import matplotlib.pyplot as plt

/home/padawan/anaconda3/envs/cajal2019/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#Select data
dat_dir = '/data/rw_eleves/Cajal-Morphometry2019/ds001748-download/'
sub_glob = sorted(glob.glob(op.join(dat_dir, 'sub*')))
sub_dirs = [op.basename(p) for p in sub_glob]
sub_dirs = sub_dirs[:3] #just take first three to try

In [3]:
# make directory and initiate workflow

!mkdir /home/padawan/Cajal2019_morphometry/Day_6
my_wf = Workflow(name="anat_processing", base_dir="/home/padawan/Day_6")


mkdir: cannot create directory ‘/home/padawan/Cajal2019_morphometry/Day_6’: File exists


In [4]:
# subject node
subjects = Node(IdentityInterface(fields=['subject_id']), name="subjects")
subjects.iterables = ('subject_id', sub_dirs)

In [5]:
# Node to grab the data
dg_node = Node(DataGrabber(infields=['subject_id'],
                      outfields=['T1']),
          name='datagrabber')

# Location of the dataset folder
dg_node.inputs.base_directory = '/data/rw_eleves/Cajal-Morphometry2019/ds001748-download/'
dg_node.inputs.template_args = {'T1': [['subject_id', 'subject_id', 'T1w']]}
dg_node.inputs.template = '%s/anat/%s_%s.nii.gz'
dg_node.inputs.sort_filelist = True

my_wf.connect(subjects, "subject_id", dg_node, "subject_id")


In [6]:
#Bias Field Correction doesn't work here so going to use FAST to get a bias corrected image

BiasCorrectNode = Node(FAST(), name = "BiasCorrect")
BiasCorrectNode.inputs.output_biascorrected = True
BiasCorrectNode.inputs.iters_afterbias = 1
BiasCorrectNode.inputs.no_pve = True
BiasCorrectNode.inputs.segments = False
BiasCorrectNode.inputs.probability_maps = False
my_wf.connect(dg_node, "T1", BiasCorrectNode, "in_files")


In [7]:
#betT1_node = Node(BET(frac = 0.3), name = "betT1")
#betT1_node.inputs.mask = True

#my_wf.connect(BiasCorrectNode,"restored_image", betT1_node, "in_file")

#BET doesn't give a good skull strip when using only the T1 image 

In [8]:
#Another way to do a skull strip is to do a linear transform of your t1 image to common space using FLIRT
#use the affine from here to then do a more fine tuned non-linear transform using FNIRT 
#and then use the inverse of this using invWarp

FLIRT_node = Node(FLIRT(), name = "LinTransform")
FLIRT_node.inputs.reference = '/modules/apps/fsl/6.0.1/data/standard/MNI152_T1_1mm.nii.gz'

my_wf.connect(BiasCorrectNode, "restored_image", FLIRT_node, "in_file")

FNIRT_node = Node(FNIRT(), name = "NonLinTransform")
FNIRT_node.inputs.ref_file = '/modules/apps/fsl/6.0.1/data/standard/MNI152_T1_1mm.nii.gz'
FNIRT_node.inputs.fieldcoeff_file = True
my_wf.connect(BiasCorrectNode, "restored_image", FNIRT_node, "in_file")
my_wf.connect(FLIRT_node, "out_matrix_file", FNIRT_node, "affine_file")

InvWarp_node = Node(InvWarp(), name = "InvertedImage")
my_wf.connect(FNIRT_node, "fieldcoeff_file", InvWarp_node, "warp")
my_wf.connect(BiasCorrectNode,"restored_image", InvWarp_node, "reference")


In [9]:
my_wf.run()

190917-09:19:23,27 nipype.workflow INFO:
	 Workflow anat_processing settings: ['check', 'execution', 'logging', 'monitoring']
190917-09:19:23,39 nipype.workflow INFO:
	 Running serially.
190917-09:19:23,40 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.datagrabber" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult03/datagrabber".
190917-09:19:23,43 nipype.workflow INFO:
	 [Node] Running "datagrabber" ("nipype.interfaces.io.DataGrabber")
190917-09:19:23,48 nipype.workflow INFO:
	 [Node] Finished "anat_processing.datagrabber".
190917-09:19:23,49 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.BiasCorrect" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult03/BiasCorrect".
190917-09:19:23,51 nipype.workflow INFO:
	 [Node] Cached "anat_processing.BiasCorrect" - collecting precomputed outputs
190917-09:19:23,52 nipype.workflow INFO:
	 [Node] "anat_processing.BiasCorrect" found cached.
190917-09:19:23,52 nipype.workflow INFO:
	 [Node] Setting-up "

In [10]:
#Apply Warp to mask

ApplyWarp_node = Node(ApplyWarp(), name = 'warp_mask')
ApplyWarp_node.inputs.in_file = '/modules/apps/fsl/6.0.1/data/standard/MNI152_T1_1mm_brain_mask.nii.gz'
my_wf.connect(BiasCorrectNode, "restored_image", ApplyWarp_node, "ref_file")
my_wf.connect(InvWarp_node, "inverse_warp", ApplyWarp_node, "field_file")



In [11]:
#Apply new warped mask to restored image

ApplyMaskNode = Node(ApplyMask(), "ApplyWarpedMask")
my_wf.connect(ApplyWarp_node, "out_file", ApplyMaskNode, "mask_file")
my_wf.connect(BiasCorrectNode, "restored_image", ApplyMaskNode, "in_file")

In [12]:
my_wf.run()

190917-09:19:24,709 nipype.workflow INFO:
	 Workflow anat_processing settings: ['check', 'execution', 'logging', 'monitoring']
190917-09:19:24,723 nipype.workflow INFO:
	 Running serially.
190917-09:19:24,724 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.datagrabber" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult03/datagrabber".
190917-09:19:24,728 nipype.workflow INFO:
	 [Node] Running "datagrabber" ("nipype.interfaces.io.DataGrabber")
190917-09:19:24,731 nipype.workflow INFO:
	 [Node] Finished "anat_processing.datagrabber".
190917-09:19:24,732 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.BiasCorrect" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult03/BiasCorrect".
190917-09:19:24,735 nipype.workflow INFO:
	 [Node] Cached "anat_processing.BiasCorrect" - collecting precomputed outputs
190917-09:19:24,735 nipype.workflow INFO:
	 [Node] "anat_processing.BiasCorrect" found cached.
190917-09:19:24,736 nipype.workflow INFO:
	 [Node] Set

190917-09:59:21,203 nipype.workflow INFO:
	 [Node] Running "datagrabber" ("nipype.interfaces.io.DataGrabber")
190917-09:59:21,208 nipype.workflow INFO:
	 [Node] Finished "anat_processing.datagrabber".
190917-09:59:21,209 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.BiasCorrect" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult01/BiasCorrect".
190917-09:59:21,210 nipype.workflow INFO:
	 [Node] Cached "anat_processing.BiasCorrect" - collecting precomputed outputs
190917-09:59:21,211 nipype.workflow INFO:
	 [Node] "anat_processing.BiasCorrect" found cached.
190917-09:59:21,211 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.LinTransform" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult01/LinTransform".
190917-09:59:21,213 nipype.workflow INFO:
	 [Node] Cached "anat_processing.LinTransform" - collecting precomputed outputs
190917-09:59:21,214 nipype.workflow INFO:
	 [Node] "anat_processing.LinTransform" found cached.
190917-09:59:21,214 nip

In [13]:
#Perform tissue segmentation

FAST_node = Node(FAST(), name = "fast")
FAST_node.inputs.img_type = 1
FAST_node.inputs.number_classes = 3

my_wf.connect(ApplyMaskNode, "out_file", FAST_node, "in_files")


In [14]:
my_wf.run()

190917-10:18:58,322 nipype.workflow INFO:
	 Workflow anat_processing settings: ['check', 'execution', 'logging', 'monitoring']
190917-10:18:58,337 nipype.workflow INFO:
	 Running serially.
190917-10:18:58,338 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.datagrabber" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult03/datagrabber".
190917-10:18:58,342 nipype.workflow INFO:
	 [Node] Running "datagrabber" ("nipype.interfaces.io.DataGrabber")
190917-10:18:58,346 nipype.workflow INFO:
	 [Node] Finished "anat_processing.datagrabber".
190917-10:18:58,347 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.BiasCorrect" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult03/BiasCorrect".
190917-10:18:58,349 nipype.workflow INFO:
	 [Node] Cached "anat_processing.BiasCorrect" - collecting precomputed outputs
190917-10:18:58,350 nipype.workflow INFO:
	 [Node] "anat_processing.BiasCorrect" found cached.
190917-10:18:58,350 nipype.workflow INFO:
	 [Node] Set

190917-10:24:45,200 nipype.workflow INFO:
	 [Node] "anat_processing.BiasCorrect" found cached.
190917-10:24:45,201 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.LinTransform" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult01/LinTransform".
190917-10:24:45,203 nipype.workflow INFO:
	 [Node] Cached "anat_processing.LinTransform" - collecting precomputed outputs
190917-10:24:45,203 nipype.workflow INFO:
	 [Node] "anat_processing.LinTransform" found cached.
190917-10:24:45,203 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.NonLinTransform" in "/home/padawan/Day_6/anat_processing/_subject_id_sub-adult01/NonLinTransform".
190917-10:24:45,206 nipype.workflow INFO:
	 [Node] Cached "anat_processing.NonLinTransform" - collecting precomputed outputs
190917-10:24:45,206 nipype.workflow INFO:
	 [Node] "anat_processing.NonLinTransform" found cached.
190917-10:24:45,207 nipype.workflow INFO:
	 [Node] Setting-up "anat_processing.InvertedImage" in "/home/padawan/D

In [ ]:
from nipype.interfaces.io import DataSink

sink = Node(DataSink(), name='sink_result')
sink.inputs.base_directory = '/home/padawan/Day_6/data_sink'

# to create one folder per subject
my_wf.connect(subjects, 'subject_id', sink, 'container')

# you still have to connect all the nodes whose results you want to get
my_wf.connect(BiasCorrectNode, "restored_image", sink, "T1_restored")
my_wf.connect(FAST_node, "partial_volume_files", sink, "PVE_files")
my_wf.connect(FAST_node, "restored_image", sink, "Bias_Corrected_T1")
my_wf.connect(FAST_node, "tissue_class_files", sink, "Tissue_Class_Files")